In [1]:
from model import param, params, logit, survey
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [2]:
# load data
sago = survey()
sago.load()
attr_labels = ['liv_rpa', 'liv_ri', 'liv_chsld', 'sup_pri', 'sup_npf', 'svc_inf', 'svc_avq', 'svc_avd', 'cons', 'wait']
#attr_labels = ['liv_rpa', 'liv_ri', 'liv_chsld', 'svc_inf', 'svc_avq', 'svc_avd', 'cons']
attributes = sago.extract_attributes(attr=attr_labels)
cs = sago.extract_choices()

                count         mean          std         min          25%  \
choice         9684.0     2.090768     0.832150    1.000000     1.000000   
svc_inf1       9684.0     6.584761     6.336579    0.000000     2.165000   
svc_inf2       9684.0     6.653507     6.329680    0.000000     2.165000   
svc_inf3       9684.0    11.158558     8.906957    0.000000     2.165000   
svc_avq1       9684.0    22.337573    22.594798    1.082500     6.495000   
svc_avq2       9684.0    22.524808    22.834558    1.082500     6.495000   
svc_avq3       9684.0    38.792601    32.205421    1.082500     7.577500   
svc_avd1       9684.0    29.626563    19.776153    2.165000    12.990000   
svc_avd2       9684.0    28.887905    19.785529    2.165000    11.907500   
svc_avd3       9684.0    44.996297    22.638309    2.165000    23.815000   
need_inf       9684.0    12.666055     7.650531    3.247500     8.660000   
need_avq       9684.0    42.774623    28.525789    8.660000    17.590625   
need_avd    

In [3]:
len(sago.data)

9684

In [4]:
len(sago.data)/4

2421.0

In [5]:
df_stat = pd.DataFrame(index=attr_labels,columns=['mean','std','min','max'])
for s in attr_labels:    
    df_stat.loc[s,'mean'] = attributes[s].mean()
    df_stat.loc[s,'std'] = attributes[s].std()
    df_stat.loc[s,'min'] = attributes[s].min()
    df_stat.loc[s,'max'] = attributes[s].max()

In [6]:
df_stat

,mean,std,min,max
liv_rpa,0.41381,0.492515,0.0,1.0
liv_ri,0.0559,0.229728,0.0,1.0
liv_chsld,0.117582,0.322113,0.0,1.0
sup_pri,0.335399,0.47213,0.0,1.0
sup_npf,0.274783,0.446405,0.0,1.0
svc_inf,1.608953,1.063622,0.0,3.480471
svc_avq,2.711765,1.233658,0.079273,4.51009
svc_avd,3.220707,0.913951,0.77242,4.156811
cons,3.084672,1.298418,0.0,5.521461
wait,0.250803,0.291266,0.0,1.333333


In [7]:
# initialize parameters
init_pars = params()
#shifter_labels = ['constant']
shifter_labels = ['constant','needs_level']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income','age_norm']
#shifter_labels = ['constant','needs_level','prob_help_adl','female','couple','college','univ']
#shifter_labels = ['constant','needs_level','prob_help_adl','female','couple','college','univ','ln_income']


for s in ['liv_rpa','liv_ri','liv_chsld']:
     for h in shifter_labels:
          if h=='ln_income':
               p = param(s,'ln_income',value = 0,free=False)
          else:
               p = param(s,h,value = 0)
          init_pars.add_param(p)

for s in ['sup_pri','sup_npf']:
     for h in shifter_labels:
          if h=='constant':
               p = param(s,'constant',value = -2)
          elif h=='ln_income':
               p = param(s,'ln_income',value = 0)
          else :
               #p = param(s,h,value = 0, free=False)
               p = param(s,h,value = 0)
          init_pars.add_param(p)


for h in shifter_labels:
     if h=='constant':
          p = param('svc_inf','constant',value = -2)
     elif h=='ln_income':
          p = param('svc_inf','ln_income',value = 0,free=False)
     else :
          p = param('svc_inf',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avq','constant',value = -2)
     elif h=='ln_income':
          p = param('svc_avq','ln_income',value = 0,free=False)
     else :
          p = param('svc_avq',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avd','constant',value = -2)
     elif h=='ln_income':
          p = param('svc_avd','ln_income',value = 0,free=False)
     else :
          p = param('svc_avd',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('cons',h,value = -2)
     elif h=='ln_income':
          p = param('cons','ln_income',value = 0,free=False)
     else :
          p = param('cons',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('wait',h,value = -2)
     else :
          #p = param('wait',h,value = 0,free=False)
          p = param('wait',h,value = 0)
     init_pars.add_param(p)

shifters = sago.extract_shifters(shifter_labels)     
n_attr = len(sago.attr_labels) 

In [8]:
model = logit(with_uh = False)
model.set_params(init_pars)
for s in sago.attr_labels:  
    model.set_shifters(s,shifters)
model.set_choices(cs)

ctr = [0,0,0,0,0,1,1,1,1,-1]
#ctr = [0,0,0,1,1,1,1]
ctrs = dict(zip(sago.attr_labels,ctr))
model.set_attributes(attributes, sago.attr_labels, ctrs)

In [9]:
model.estimate(scn_name = 'pre_uh_need_level')

iter =  0 , loglike =  -4.65024559580715
current theta:
[-0.24534637 -0.09783077 -0.09132254 -0.07164568 -0.02686024 -0.01241994
 -1.37238514  0.38214254 -1.47776788  0.28284961 -2.00792581 -0.00934172
 -2.00976342 -0.00990474 -1.99617374 -0.00439858 -1.98007137  0.01471055
 -1.98974681  0.0071525 ]
iter =  1 , loglike =  -4.252387327254142
current theta:
[-0.56636436 -0.17592404 -0.20903691 -0.16173276 -0.18389139 -0.11194495
 -0.3463122   0.99403938 -0.31248888  0.90475586 -2.02820311 -0.03254283
 -2.03632671 -0.03556936 -1.98426153 -0.01371291 -1.91646934  0.06194908
 -1.96567321  0.02350558]
iter =  2 , loglike =  -4.18782310609705
current theta:
[-0.52564916 -0.16543494 -0.1026403  -0.07608263 -0.11695184 -0.05461863
 -0.60727547  0.82208087 -0.55844683  0.74755757 -2.00641023 -0.01558035
 -2.01579152 -0.01967542 -1.95784214  0.00585568 -1.90893286  0.06841244
 -1.9770721   0.01406388]
iter =  3 , loglike =  -4.179263069874217
current theta:
[-0.51085996 -0.13819417 -0.04054622 -0

In [10]:
init_pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-1.004589,0.069332,-14.489446,True
1,liv_rpa,needs_level,0.820483,0.119233,6.881323,True
2,liv_ri,constant,-0.291653,0.340676,-0.8561,True
3,liv_ri,needs_level,0.678686,0.43505,1.560019,True
4,liv_chsld,constant,-1.591267,0.262737,-6.056494,True
5,liv_chsld,needs_level,2.041836,0.338005,6.040846,True
6,sup_pri,constant,-0.062988,0.079531,-0.791996,True
7,sup_pri,needs_level,0.069744,0.125262,0.556784,True
8,sup_npf,constant,0.039241,0.082777,0.474065,True
9,sup_npf,needs_level,-0.066129,0.13972,-0.473295,True


In [15]:
# initialize parameters
init_pars = params()

shifter_labels = ['constant','needs_level']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','prob_adl','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','med_adl_prob','high_adl_prob','adl_miss']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','ln_income']
#shifter_labels = ['constant','needs_level','prob_help_adl','help_adl_miss','adl_miss','ln_income','age_norm']

for s in ['liv_rpa','liv_ri','liv_chsld']:
     for h in shifter_labels:
          if h=='ln_income':
               p = param(s,'ln_income',value = 0,free=False)
          else:
               p = param(s,h,value = 0)
          init_pars.add_param(p)

for s in ['sup_pri','sup_npf']:
     for h in shifter_labels:
          if h=='constant':
               p = param(s,'constant',value = -2)
          elif h=='ln_income':
               p = param(s,'ln_income',value = 0)
          else :
               #p = param(s,h,value = 0, free=False)
               p = param(s,h,value = 0)
          init_pars.add_param(p)


for h in shifter_labels:
     if h=='constant':
          p = param('svc_inf','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_inf','ln_income',value = 0,free=False)
     else :
          p = param('svc_inf',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avq','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_avq','ln_income',value = 0,free=False)
     else :
          p = param('svc_avq',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('svc_avd','constant',value = -2)
     #elif h=='ln_income':
     #     p = param('svc_avd','ln_income',value = 0,free=False)
     else :
          p = param('svc_avd',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('cons',h,value = -2)
     else :
          p = param('cons',h,value = 0)
     init_pars.add_param(p)

for h in shifter_labels:
     if h=='constant':
          p = param('wait',h,value = -2)
     else :
          #p = param('wait',h,value = 0,free=False)
          p = param('wait',h,value = 0)
     init_pars.add_param(p)
shifters = sago.extract_shifters(shifter_labels)     
n_attr = len(sago.attr_labels) 

In [16]:
init_pars.restart_from('pre_uh_help_adl')
for d in range(n_attr):
     for u in range(d,n_attr):
      if (d==u) & (u in [0,1,2,5,6,7,9]):
            p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
      elif (d==u) & (u in [3,4,8]):
            p = param('uh','ln_sig_'+str(d)+str(u), value = -10,free=False)
            #p = param('uh','ln_sig_'+str(d)+str(u), value = np.log(0.1),free=True)
      elif (d in [5,6]) & (u in [6,7]) & (d!=u):
            p = param('uh','rho_'+str(d)+str(u), value = 0,free=True)
      else:
            p = param('uh','rho_'+str(d)+str(u),value = 0,free=False)
      init_pars.add_param(p)

#init_pars.restart_from('uh_prob_help_income_corr')
#init_pars.pars[90].value = np.log(0.2)
#init_pars.pars[95].value = np.log(0.2)
#init_pars.pars[99].value = np.log(0.2)
#init_pars.pars[104].value = np.log(0.2)
#init_pars.pars[67].value = np.log(0.000001)
#init_pars.pars[74].value = np.log(0.000001)
#init_pars.pars[80].value = np.log(0.2)
#init_pars.pars[85].value = np.log(0.2)
#init_pars.pars[74].value = np.log(0.2)
#init_pars.pars[77].value = np.log(0.2)
#init_pars.pars[84].value = np.log(0.2)


    Unnamed: 0      group          label     value    stderr     tvalue   free
0            0    liv_rpa       constant -0.937223  0.073250 -12.794800   True
1            1    liv_rpa    needs_level  0.834417  0.119846   6.962393   True
2            2    liv_rpa  prob_help_adl -0.217812  0.078740  -2.766222   True
3            3    liv_rpa  help_adl_miss -0.161200  0.067820  -2.376898   True
4            4     liv_ri       constant -0.285052  0.331543  -0.859775   True
5            5     liv_ri    needs_level  0.777621  0.415638   1.870909   True
6            6     liv_ri  prob_help_adl -0.233827  0.180689  -1.294088   True
7            7     liv_ri  help_adl_miss -0.211045  0.166761  -1.265554   True
8            8  liv_chsld       constant -1.500582  0.253234  -5.925673   True
9            9  liv_chsld    needs_level  2.178161  0.318364   6.841728   True
10          10  liv_chsld  prob_help_adl -0.455118  0.125384  -3.629792   True
11          11  liv_chsld  help_adl_miss -0.579276  

In [17]:
init_pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-0.937223,NaN,NaN,True
1,liv_rpa,needs_level,0.834417,NaN,NaN,True
2,liv_ri,constant,-0.285052,NaN,NaN,True
3,liv_ri,needs_level,0.777621,NaN,NaN,True
4,liv_chsld,constant,-1.500582,NaN,NaN,True
5,liv_chsld,needs_level,2.178161,NaN,NaN,True
6,sup_pri,constant,-0.022095,NaN,NaN,True
7,sup_pri,needs_level,0.0,NaN,NaN,True
8,sup_npf,constant,0.006757,NaN,NaN,True
9,sup_npf,needs_level,0.0,NaN,NaN,True


In [18]:
model = logit(with_uh = True)
#model = logit(with_uh = False)
model.set_params(init_pars)
for s in sago.attr_labels:  
    model.set_shifters(s,shifters)
model.set_choices(cs)

ctr = [0,0,0,0,0,1,1,1,1,-1]
#ctr = [0,0,0,1,1,1,1]
ctrs = dict(zip(sago.attr_labels,ctr))
model.set_attributes(attributes, sago.attr_labels, ctrs)
model.set_draws(n_d = 25)

In [19]:
model.estimate(scn_name ='uh_need_level',algorithm='BFGS')
#model.estimate(scn_name ='uh1',algorithm='BFGS')

iter =  0 , loglike =  -4.11386156705449
current theta:
[-9.55045475e-01  8.30095657e-01 -2.87496487e-01  7.75707116e-01
 -1.54796351e+00  2.14059587e+00 -2.95348591e-02 -2.58541107e-03
  2.28468768e-02  1.01391077e-02 -7.32771875e-01 -1.86766380e+00
 -1.19805008e+00 -9.19766829e-01 -7.78723794e-01 -1.34319303e+00
 -6.55208331e-01 -7.45574210e-01 -1.46039869e+00  4.27561998e-03
 -2.30123481e+00 -2.30217036e+00 -2.30115869e+00 -2.30251977e+00
  2.50339508e-06  1.77860260e-04 -2.30268463e+00  1.63316727e-04
 -2.30261621e+00 -2.30245057e+00]
iter =  1 , loglike =  -4.112074835368547
current theta:
[-9.94549118e-01  8.13967404e-01 -2.91788005e-01  7.72288257e-01
 -1.59775644e+00  2.10152814e+00 -1.96645264e-02  8.13075713e-03
  1.78404385e-02  5.74531623e-03 -7.29257509e-01 -1.86782620e+00
 -1.21312548e+00 -9.29351324e-01 -7.74476895e-01 -1.34463370e+00
 -6.34336114e-01 -7.30954770e-01 -1.45440134e+00  8.77131975e-03
 -2.29848082e+00 -2.30132757e+00 -2.29825171e+00 -2.30233741e+00
  9.2529

LinAlgError: Matrix is not positive definite

In [23]:
model.pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-0.296142,NaN,NaN,True
1,liv_rpa,needs_level,-0.757659,NaN,NaN,True
2,liv_ri,constant,-1.901234,NaN,NaN,True
3,liv_ri,needs_level,1.260157,NaN,NaN,True
4,liv_chsld,constant,-2.324706,NaN,NaN,True
5,liv_chsld,needs_level,2.313465,NaN,NaN,True
6,sup_pri,constant,-0.419491,NaN,NaN,True
7,sup_pri,needs_level,0.517336,NaN,NaN,True
8,sup_npf,constant,-0.217379,NaN,NaN,True
9,sup_npf,needs_level,0.103671,NaN,NaN,True


In [ ]:
model.pars.Sig[0][9]

0.0

In [21]:
df_sig = pd.DataFrame(index=range(10),columns=range(10))
for i in range(10):
    for j in range(10):
            df_sig.loc[i,j] = model.pars.Sig[i][j]

df_sig


,0,1,2,3,4,5,6,7,8,9
0,0.545057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.646933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,5.235293,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.019434,0.000507,797.201908,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.000507,0.000023,22.350625,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,797.201908,22.350625,32732919.783847,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.436195


In [22]:
np.linalg.cholesky(model.pars.Sig)

LinAlgError: Matrix is not positive definite

In [ ]:
init_pars.table()

,group,label,value,stderr,tvalue,free
0,liv_rpa,constant,-28.240274,NaN,NaN,True
1,liv_rpa,needs_level,85.357672,NaN,NaN,True
2,liv_rpa,prob_help_adl,7.547916,NaN,NaN,True
3,liv_rpa,help_adl_miss,15.323811,NaN,NaN,True
4,liv_rpa,adl_miss,-65.814393,NaN,NaN,True
5,liv_rpa,ln_income,0.0,NaN,NaN,False
6,liv_ri,constant,66.654349,NaN,NaN,True
7,liv_ri,needs_level,-46.091705,NaN,NaN,True
8,liv_ri,prob_help_adl,-6.349881,NaN,NaN,True
9,liv_ri,help_adl_miss,-4.622242,NaN,NaN,True
